In [1]:
%pip install python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt langchain-chroma

  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
  Using cached langchain-0.3.14-py3-none-any.whl (1.0 MB)
  Using cached langchain_openai-0.2.14-py3-none-any.whl (50 kB)
  Using cached langchain_community-0.3.14-py3-none-any.whl (2.5 MB)
  Using cached langchain_text_splitters-0.3.4-py3-none-any.whl (27 kB)
  Using cached docx2txt-0.8.tar.gz (2.8 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached langchain_chroma-0.1.4-py3-none-any.whl (10 kB)
  Using cached aiohttp-3.11.11-cp310-cp310-win_amd64.whl (442 kB)
  Using cached langsmith-0.2.10-py3-none-any.whl (326 kB)
  Using cached langchain_core-0.3.29-py3-none-any.whl (411 kB)
  Using cached SQLAlchemy-2.0.36-cp310-cp310-win_amd64.whl (2.1 MB)
  Using cached tenacity-9.0.0-py3-none-any.whl (28 kB)
  Using cached pydantic-2.10.4-py3-none-any.whl (431 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
  Using cached openai-1.59.3-py3-none-any.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 5.29.2 which is incompatible.
tensorboard 2.11.2 requires protobuf<4,>=3.9.2, but you have protobuf 5.29.2 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
%pip install --upgrade --quite \
    langchain-pinecone \
    langchain-openai \
    langchain

Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\kimdo\AppData\Local\pypoetry\Cache\virtualenvs\rag-llm어플리케이션실습-dgGHJXyp-py3.10\Scripts\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\kimdo\AppData\Local\pypoetry\Cache\virtualenvs\rag-llm어플리케이션실습-dgGHJXyp-py3.10\Scripts\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\kimdo\AppData\Local\pypoetry\Cache\virtualenvs\rag-llm어플리케이션실습-dgGHJXyp-py3.10\Scripts\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\kimdo\AppData\Local\pypoetry\Cache\virtualenvs\rag-llm어플리케이션실습-dgGHJXyp-py3.10\Scripts\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\kimdo\AppData\Local\pypoetry\Cache\virtualenvs\rag-llm어플리케이션실습-dgGHJXyp-py3.10\Scripts\python.exe -m pip install [options] <archive url/path> ...

no such option: --quite


In [4]:
from dotenv import load_dotenv
from langchain import hub
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import RetrievalQA

load_dotenv()

index_name = 'tax-markdown-table-index'

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")
database = PineconeVectorStore.from_existing_index(embedding=embedding, index_name=index_name)

retriever = database.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
  llm,
  retriever=retriever,
  chain_type_kwargs={"prompt": prompt}
)

dictionary = ["사람을 나타내는 표현 -> 거주자"]

custom_prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    사전: {dictionary}

    질문: {{question}}
""")

dictionary_chain = custom_prompt | llm | StrOutputParser()

tax_chain = {"query": dictionary_chain} | qa_chain

c:\Users\kimdo\AppData\Local\pypoetry\Cache\virtualenvs\rag-llm어플리케이션실습-dgGHJXyp-py3.10\lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [5]:
ai_response = tax_chain.invoke({"question": "연봉 5000만원 직작인의 소득세를 알려주세요"})
print(ai_response)

{'query': '변경된 질문: 연봉 5000만원 거주자의 소득세를 알려주세요', 'result': '5000만원 초과 8,800만원 이하의 종합소득 과세표준에는 624만원 + (5,000만원을 초과하는 금액의 24%) 세율이 적용됩니다. 변경된 질문: 연봉 5000만원 거주자의 소득세를 알려주세요. 따라서 5000만원 연봉의 거주자의 세금은 해당 세율에 따라 계산됩니다.'}
